In [175]:

import pandas as pd
from scipy.stats import linregress,theilslopes
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy.interpolate import lagrange

* entry: {year: (investment,AQI/,COD,CO2/,GDP/,CITYLIZE/)}

In [176]:
provinces = {
    '上海市': ['上海市'],
    '江苏省': ['南京市', '无锡市', '徐州市', '常州市', '苏州市', '南通市', '连云港市', '淮安市', '盐城市', '扬州市', '镇江市', '泰州市', '宿迁市'],
    '浙江省': ['杭州市', '宁波市', '温州市', '嘉兴市', '湖州市', '绍兴市', '金华市', '衢州市', '舟山市', '台州市', '丽水市'],
    '安徽省': ['合肥市', '芜湖市', '蚌埠市', '淮南市', '马鞍山市', '淮北市', '铜陵市', '安庆市', '黄山市', '滁州市', '阜阳市', '宿州市', '六安市', '亳州市', '池州市', '宣城市']
}
# 定义希望保存的列名
desired_columns = ['month', 'AQI']
collected_data = r"datasets\datasets_data\collected"
dataset_store = f"{collected_data}\\processed"

In [139]:
# air condition
gdp_data = pd.ExcelFile(r'datasets\datasets_data\collected\air_condition.xlsx')

# 遍历每个省份
for province, cities in provinces.items():
    # 存储当前省份的所有数据
    province_data = pd.DataFrame(columns=['月份', 'AQI'])
    
    # 遍历该省份的每个城市
    for city in cities:
        if city in gdp_data.sheet_names:
            # 读取当前城市的数据
            df = pd.read_excel(gdp_data, sheet_name=city)
            
            # 选择需要的列
            selected_columns = df[['月份', 'AQI']]
            
            # 将月份列转换为日期类型
            selected_columns['月份'] = pd.to_datetime(selected_columns['月份'], format='%b-%y')
            
            # 将当前城市的数据添加到省份数据中
            province_data = pd.concat([province_data, selected_columns], ignore_index=True)
    
    # 按月份分组,计算AQI的平均值,并填充缺失值为0
    province_monthly_avg = province_data.groupby(pd.Grouper(key='月份', freq='M'))['AQI'].mean().fillna(0).reset_index()
    
    # 按日期排序
    province_monthly_avg = province_monthly_avg.sort_values('月份')
    
    # 保存省份月度平均数据到csv文件
    province_monthly_avg.to_csv(f'{province}_monthly_avg.csv', index=False)
    
    # 按年份分组,计算AQI的平均值
    province_yearly_avg = province_monthly_avg.groupby(province_monthly_avg['月份'].dt.year)['AQI'].mean().reset_index()
    province_yearly_avg.columns = ['年份', 'AQI']
    
    # 使用线性回归将数据外推到2000年
    x = province_yearly_avg['年份'].values.ravel()
    y = province_yearly_avg['AQI'].values.ravel()
    lin_result = linregress(x, y,alternative='less')
    
    # 生成2000年到最后年份的年份列表
    min_year = province_yearly_avg['年份'].min()
    years = range(2000, min_year-1, 1)
    
    # 计算外推的AQI值
    extrapolated_aqi = [lin_result.slope * year + lin_result.intercept for year in years]
    
    # 创建外推数据的DataFrame
    extrapolated_data = pd.DataFrame({'年份': years, 'AQI': extrapolated_aqi})
    
    # 将外推数据与原始数据合并
    province_yearly_avg = pd.concat([extrapolated_data, province_yearly_avg],axis=0, ignore_index=True)
    
    
    # 保存省份年度平均数据到csv文件
    province_yearly_avg.to_csv(f'{dataset_store}\\aqi\\{province}_yearly_aircondition.csv', index=False)

print("数据处理完成!")

C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['月份'] = pd.to_datetime(selected_columns['月份'], format='%b-%y')
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  province_data = pd.concat([province_data, selected_columns], ignore_index=True)
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:25: FutureWarning: 'M' is deprecated and will be remov

数据处理完成!


C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['月份'] = pd.to_datetime(selected_columns['月份'], format='%b-%y')
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['月份'] = pd.to_datetime(selected_columns['月份'], format='%b-%y')
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1800158117.py:19: SettingWithCopyWarning: 
A value is trying to be set on 

In [4]:
citylize = pd.read_csv(f"{collected_data}\\citylize.csv")[['时间','rate']]
citylize['时间'] = pd.to_datetime(citylize['时间'])

In [138]:
# gdp
excel_file = pd.ExcelFile(r'datasets\datasets_data\collected\gdp.xlsx')
# 定义拟合函数
def logistic_func(x, a, b, c):
    return c / (1 + a * np.exp(-b*x))
    
# 遍历每个省份
for province, cities in provinces.items():
    # 存储当前省份的所有数据
    province_data = []
    
    # 遍历该省份的每个城市
    for city in cities:
        if city in excel_file.sheet_names:
            # 读取当前城市的数据
            df = pd.read_excel(excel_file, sheet_name=city)
            
            # 选择需要的列
            selected_columns = df[['年份', '总量（亿元）']]
            
            # 将城市名称添加为新列
            selected_columns['城市'] = city
            
            # 将省份名称添加为新列
            selected_columns['province'] = province
            
            # 处理年份列的异常值
            selected_columns['年份'] = selected_columns['年份'].astype(str)
            selected_columns['年份'] = selected_columns['年份'].str.extract(r'(\d{4})', expand=False)
            selected_columns['年份'] = pd.to_datetime(selected_columns['年份'], errors='coerce')
            selected_columns.drop_duplicates()
            
            # 处理总量列的异常值
            selected_columns['总量（亿元）'] = selected_columns['总量（亿元）'].astype(str).str.strip().replace(' ', '').replace('', pd.NA)
            selected_columns['总量（亿元）'] = pd.to_numeric(selected_columns['总量（亿元）'], errors='coerce')
            
            # 删除年份或总量为空的行
            selected_columns = selected_columns.dropna(subset=['年份', '总量（亿元）'])
            
            # 将当前城市的数据添加到省份数据列表中
            province_data.append(selected_columns)
    
    # 将省份数据列表合并为一个DataFrame
    province_df = pd.concat(province_data, ignore_index=True)
    
    # 将年份列转换为数值类型
    province_df['年份'] = province_df['年份'].apply(lambda y:y.year)
    
    # 按省份和年份分组,计算"总量(亿元)"的总和
    province_df = province_df.groupby(['province', '年份'])['总量（亿元）'].sum().reset_index()
    
    # 按省份和城市分组
    grouped = province_df.groupby('province')
    
    # 对每个城市进行插值和外推
    for (province), group in grouped:
        # 获取当前城市的年份和总量数据
        x = group['年份'].values
        y = group['总量（亿元）'].values
        
        # 创建插值函数
        f = interp1d(x, y, kind='linear', fill_value='extrapolate')
        
        # 生成完整的年份范围
        all_years = range(2000, max(x) + 1)
        
        # 使用插值函数计算缺失值和外推值
        interpolated_values = f(all_years)
        
        # 将插值结果添加到原始数据中
        for year, value in zip(all_years, interpolated_values):
            # province_df.loc[(province_df['province'] == province)  & (province_df['年份'] == year), '总量（亿元）'] = value
            if np.int64(year) not in np.int64(province_df['年份']):
                province_df = pd.concat([province_df,pd.DataFrame([{'province':province,'年份':year,'总量（亿元）':value}])],ignore_index=True)
    
    # 保存省份年度总和数据到csv文件
    province_df.to_csv(f'{dataset_store}\\gdp\\{province}_gdp_yearly_sum.csv', index=False)

print("数据处理完成!")



C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['城市'] = city
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['province'] = province
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

数据处理完成!


C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['城市'] = city
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['province'] = province
C:\Users\19920\AppData\Local\Temp\ipykernel_33340\1450516094.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [182]:
carbon_emission = pd.read_excel(f"{collected_data}\\carbon_emission.xlsx",sheet_name='emission vector')
carbon_emission = carbon_emission[['city','year','emission']]
provinces_no_suffix = {
    '上海': ['上海'],
    '江苏': ['南京', '无锡', '徐州', '常州', '苏州', '南通', '连云港', '淮安', '盐城', '扬州', '镇江', '泰州', '宿迁'],
    '浙江': ['杭州', '宁波', '温州', '嘉兴', '湖州', '绍兴', '金华', '衢州', '舟山', '台州', '丽水'],
    '安徽': ['合肥', '芜湖', '蚌埠', '淮南', '马鞍山', '淮北', '铜陵', '安庆', '黄山', '滁州', '阜阳', '宿州', '六安', '亳州', '池州', '宣城']
}

# 遍历每个省份
for province, cities in provinces_no_suffix.items():
    # 过滤出当前省份的数据
    province_data = carbon_emission[carbon_emission['city'].str.contains(province)]
    
    # 如果省份数据不为空
    if not province_data.empty:
        # 按城市分组,计算emission的总和
        city_emission = province_data.groupby('year')['emission'].sum().reset_index()
        # for item in city_emission: TODO: USE MORE RESONABLE METHOD
        #     x = item['year']
        #     y = item['emission']
        #     # 创建插值函数
        #     poly = lagrange(x, y)
        #     # 生成完整的年份范围
        #     all_years = range(2000, max(x) + 1)
        #     # 使用插值函数计算缺失值和外推值
        #     interpolated_values = [poly(year) for year in all_years]
        #     data.loc[(data['city'] == city) & (data['year'] == year), 'emission'] = value
        
        # 保存到CSV文件
        city_emission.to_csv(f'{dataset_store}\\co2\\{province}_emission.csv', index=False)

print("数据处理完成!")

数据处理完成!
